<p align="center">
<a href="https://mybinder.org/v2/gh/Giacomo20/mathnotebooks/HEAD" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open Notebook"></a>
</p>

# Networks in Jupyter notebook

Networks are commonly widespread in many areas. We will consider a well-known problem in literature, the **Shortest Path Tree** (**SPT**), to dive into some common modules for the manipulation and the representation of complex networks.
Let's start with import modules.

### Import modules

There are different options available. We will experiment some of the most used, that are:
- [NetworkX](https://networkx.org/): is a package that allows us to create, manipulate complex structures represented by a graph,
- [PyVis](https://pyvis.readthedocs.io/en/latest/): is an interactive tool to visualize and customize graphs. This package comes with some bindings to import structures created with NetworkX.

### Install environment
The following modules can be installed in a `conda` environment with only one command from the terminal:

`conda install pyvis`

In [ ]:
from pyvis import network as net
import networkx as nx
import random

### Create the graph
For example we can create a *complete* network with networkx and visualize it in our notebook.

In [ ]:
# Example 1
graph = net.Network(notebook = True)
nxgraph = nx.complete_graph(5)
graph.from_nx(nxgraph)
graph.show("complete graph.html")

There are available many generators. We use a random generator, by specifying the nodes and the edges.

In [ ]:
# Example 2: randomly generated
nxgraph = nx.dense_gnm_random_graph(8, 16)

In [ ]:
# step 1: fix nodes position in the view 
nxpos = nx.random_layout(nxgraph)
graph = net.Network(notebook = True)
graph.from_nx(nxgraph)
for node in graph.nodes:
    node['x'], node['y'] = '%.2f' %nxpos[node['id']][0], '%.2f' %nxpos[node['id']][1]

In [ ]:
# step 2: generate weights

# nx graph
for e in nxgraph.edges.data():
    e[2]['weight'] = random.random()

# graph
graph.inherit_edge_colors(False)

for edge in graph.edges:
    edge['weight'] = nxgraph.get_edge_data(edge['from'], edge['to'])['weight']
    edge['title'] = "%.2f" %edge['weight']
    edge['value'] = edge['weight']

In [ ]:
dir(graph)  # list the attributes
graph.nodes # nodes
graph.edges # edges
graph.edges[:2]

In [ ]:
graph.nodes.sort(key=lambda x:x['id'])

In [ ]:
# initialize colors
def graph_initialize_colors(graph):
    for g in graph.nodes:
        g['color'] = {}
    for e in graph.edges:
        e['color'] = {}
    # graph.nodes[len(graph.nodes) - 1]['color'] = '#ff00ff'
    graph.nodes[0]['color'] = '#0000ff'

graph_initialize_colors(graph)
graph.show("graph.html") # rerun the cell: positions are fixed!

In [ ]:
import numpy as np
import time
from IPython import display

dist = np.ones(len(graph.nodes)) * np.inf 
dist[0] = 0

pred = np.ones(len(graph.nodes), np.int32) * -1
pred[0] = 0

to_be_explored = [0]

def get_edge_data_indir(graph, u, v):
    e = ''
    for edge in graph.edges:
        if (edge['from'] == u) and (edge['to'] == v):
            e = edge
            break
        if (edge['from'] == v) and (edge['to'] == u):
            e = edge
            break
    if e == '':
        print("ERROR: not found edge (%d, %d)" %(u, v))
    return e

graph_initialize_colors(graph)
graph.show('graph.html')
time.sleep(1)

while len(to_be_explored) > 0:
    print("nodes to be explored = ", to_be_explored)
    idx_curr = to_be_explored[0]
    to_be_explored.pop(0)
    print("exploring node %d" %idx_curr)
    for edge in nx.edges(nxgraph, idx_curr):
        idx_neigh = edge[1]
        if dist[idx_neigh] == np.inf:
            to_be_explored.append(idx_neigh)
            graph.nodes[idx_neigh]['color'] = '#0000ff'

        print(edge, idx_neigh)
        if dist[idx_neigh] > dist[idx_curr] + nxgraph.get_edge_data(idx_curr, idx_neigh)['weight']:
            get_edge_data_indir(graph, idx_curr, idx_neigh)['color'] = '#0000ff'
            if pred[idx_neigh] >=0:
                get_edge_data_indir(graph, pred[idx_neigh], idx_neigh)['color'] = {}

            pred[idx_neigh] = idx_curr
            dist[idx_neigh] = dist[idx_curr] + nxgraph.get_edge_data(idx_curr, idx_neigh)['weight']

    display.display(graph.show('graph.html'))
    display.clear_output(wait = True)

    time.sleep(2)
